# **Mineria de Datos**
# **BOW Paralelizado - TF-IDF**
-----------------------------------
### **Nombres:** Antony Isaac Huaman Hermoza
### **Código:** 170434
-----------------------------------

## **Insatalamos PySpark**

In [1]:
!pip install pyspark==3.0.1 py4j==0.10.9

     |████████████████████████████████| 204.2 MB 36 kB/s 
     |████████████████████████████████| 198 kB 47.4 MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612246 sha256=fa0cb0a435da858fd131de26011b15a474fb12298956a6e96ed793240dc0e23e
  Stored in directory: /root/.cache/pip/wheels/5e/34/fa/b37b5cef503fc5148b478b2495043ba61b079120b7ff379f9b
Successfully built pyspark


## **Improtamos librerias**

In [2]:
from pyspark import SparkContext
sc = SparkContext.getOrCreate()
import math

from pyspark.sql.functions import *
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
sc =SparkContext.getOrCreate()
import os
import re
import nltk.corpus
from nltk import corpus

**BOW Paralelizado**

In [3]:
#Creamos datos para nuestro o bag of words
data=[(1,'Jake Paul se enfoca en las artes marciales mixtas'),
      (2,"El debut de Messi en 2022 tendrá que esperar"),
      (3,"El caso de Djokovic ha ido mucho más allá de una cuestión de visado"),
      (4,"La tenista checa Renata Voracova se fue de Australia")]
data

[(1, 'Jake Paul se enfoca en las artes marciales mixtas'),
 (2, 'El debut de Messi en 2022 tendrá que esperar'),
 (3, 'El caso de Djokovic ha ido mucho más allá de una cuestión de visado'),
 (4, 'La tenista checa Renata Voracova se fue de Australia')]

In [4]:
# Paraleliazmos nuestro data
lines=sc.parallelize(data)
# Recupera todos los elementos del lines
lines.collect()
map1=lines.flatMap(lambda x: [((x[0],i),1) for i in x[1].split()])
# Recupera todos los elementos del map1
map1.collect()
#Map Reduce
reduce=map1.reduceByKey(lambda x,y:x+y)
#BOG de las 4 oraciones
reduce.collect()

[((1, 'Paul'), 1),
 ((1, 'se'), 1),
 ((1, 'enfoca'), 1),
 ((1, 'las'), 1),
 ((1, 'artes'), 1),
 ((1, 'mixtas'), 1),
 ((2, 'en'), 1),
 ((2, 'que'), 1),
 ((3, 'El'), 1),
 ((3, 'de'), 3),
 ((3, 'ha'), 1),
 ((3, 'más'), 1),
 ((3, 'allá'), 1),
 ((3, 'una'), 1),
 ((4, 'tenista'), 1),
 ((4, 'Renata'), 1),
 ((4, 'Australia'), 1),
 ((1, 'Jake'), 1),
 ((1, 'en'), 1),
 ((1, 'marciales'), 1),
 ((2, 'El'), 1),
 ((2, 'debut'), 1),
 ((2, 'de'), 1),
 ((2, 'Messi'), 1),
 ((2, '2022'), 1),
 ((2, 'tendrá'), 1),
 ((2, 'esperar'), 1),
 ((3, 'caso'), 1),
 ((3, 'Djokovic'), 1),
 ((3, 'ido'), 1),
 ((3, 'mucho'), 1),
 ((3, 'cuestión'), 1),
 ((3, 'visado'), 1),
 ((4, 'La'), 1),
 ((4, 'checa'), 1),
 ((4, 'Voracova'), 1),
 ((4, 'se'), 1),
 ((4, 'fue'), 1),
 ((4, 'de'), 1)]

**TF-IDF**

In [5]:
######################################
### TF ###
######################################

# Algoritmo termino de frecuencia (TF)
tf=reduce.map(lambda x: (x[0][1],(x[0][0],x[1])))
tf.collect()

map3=reduce.map(lambda x: (x[0][1],(x[0][0],x[1],1)))
map3.collect()

map4=map3.map(lambda x:(x[0],x[1][2]))
map4.collect()

reduce2=map4.reduceByKey(lambda x,y:x+y)
reduce2.collect()

[('en', 2),
 ('que', 1),
 ('tenista', 1),
 ('Renata', 1),
 ('Australia', 1),
 ('Jake', 1),
 ('marciales', 1),
 ('caso', 1),
 ('Djokovic', 1),
 ('ido', 1),
 ('mucho', 1),
 ('cuestión', 1),
 ('visado', 1),
 ('Paul', 1),
 ('se', 2),
 ('enfoca', 1),
 ('las', 1),
 ('artes', 1),
 ('mixtas', 1),
 ('El', 2),
 ('de', 3),
 ('ha', 1),
 ('más', 1),
 ('allá', 1),
 ('una', 1),
 ('debut', 1),
 ('Messi', 1),
 ('2022', 1),
 ('tendrá', 1),
 ('esperar', 1),
 ('La', 1),
 ('checa', 1),
 ('Voracova', 1),
 ('fue', 1)]

In [6]:
######################################
### IF ###
######################################
# Algoritmo termino de frecuencia (IDF)
idf=reduce2.map(lambda x: (x[0],math.log10(len(data)/x[1])))
idf.collect()

rdd=tf.join(idf)
rdd.collect()

[('caso', ((3, 1), 0.6020599913279624)),
 ('Djokovic', ((3, 1), 0.6020599913279624)),
 ('mucho', ((3, 1), 0.6020599913279624)),
 ('cuestión', ((3, 1), 0.6020599913279624)),
 ('visado', ((3, 1), 0.6020599913279624)),
 ('se', ((1, 1), 0.3010299956639812)),
 ('se', ((4, 1), 0.3010299956639812)),
 ('enfoca', ((1, 1), 0.6020599913279624)),
 ('las', ((1, 1), 0.6020599913279624)),
 ('El', ((3, 1), 0.3010299956639812)),
 ('El', ((2, 1), 0.3010299956639812)),
 ('allá', ((3, 1), 0.6020599913279624)),
 ('una', ((3, 1), 0.6020599913279624)),
 ('debut', ((2, 1), 0.6020599913279624)),
 ('tendrá', ((2, 1), 0.6020599913279624)),
 ('esperar', ((2, 1), 0.6020599913279624)),
 ('La', ((4, 1), 0.6020599913279624)),
 ('checa', ((4, 1), 0.6020599913279624)),
 ('en', ((2, 1), 0.3010299956639812)),
 ('en', ((1, 1), 0.3010299956639812)),
 ('que', ((2, 1), 0.6020599913279624)),
 ('tenista', ((4, 1), 0.6020599913279624)),
 ('Renata', ((4, 1), 0.6020599913279624)),
 ('Australia', ((4, 1), 0.6020599913279624)),
 ('

**ENCUADRAMOS**

In [7]:
# Algoritmo termino de frecuencia (TF-IDF)
rdd=rdd.map(lambda x: (x[1][0][0],(x[0],x[1][0][1],x[1][1],x[1][0][1]*x[1][1]))).sortByKey()
rdd.collect()

spark = SparkSession(sc)
rdd=rdd.map(lambda x: (x[0],x[1][0],x[1][1],x[1][2],x[1][3]))
hasattr(rdd, "toDF")
rdd.toDF(["ID del Documento","Token","TF","IDF","TF-IDF"]).show()

+----------------+---------+---+-------------------+-------------------+
|ID del Documento|    Token| TF|                IDF|             TF-IDF|
+----------------+---------+---+-------------------+-------------------+
|               1|       se|  1| 0.3010299956639812| 0.3010299956639812|
|               1|   enfoca|  1| 0.6020599913279624| 0.6020599913279624|
|               1|      las|  1| 0.6020599913279624| 0.6020599913279624|
|               1|       en|  1| 0.3010299956639812| 0.3010299956639812|
|               1|     Jake|  1| 0.6020599913279624| 0.6020599913279624|
|               1|marciales|  1| 0.6020599913279624| 0.6020599913279624|
|               1|     Paul|  1| 0.6020599913279624| 0.6020599913279624|
|               1|    artes|  1| 0.6020599913279624| 0.6020599913279624|
|               1|   mixtas|  1| 0.6020599913279624| 0.6020599913279624|
|               2|       El|  1| 0.3010299956639812| 0.3010299956639812|
|               2|    debut|  1| 0.6020599913279624